# Copy simulation data on tigressdata/ to the astro-tigress data release file storage

In [20]:
import sys
sys.path.insert(0, "../module/") #add path for import
import numpy as np
from tigress_utils import copy_file
import read_vtk
import delete_fields_vtk

<module 'delete_fields_vtk' from '/home/munan/astro-tigress/miscellaneous/delete_fields_vtk.py'>

In [6]:
#directories
dir_new = "/tigress/munan/public_html/astro-tigress/"
dir_MHD_old = "/projects/EOSTRIKE/TIGRESS_XCO/"
dir_chem_old = "/tigress/munan/chemistry/XCO_surfd/"
dir_CO_old = "/tigress/munan/chemistry/scripts/radmc3d/output/XCO_surfd/"
dir_rad_old = "/tigress/changgoo/radps_postproc/"
#R8_2pc_rst
ivtks_R8_2pc = np.arange(290, 400, 10)
id_R8_2pc_old = "R8_2pc_rst"
chemZ_R8_old = ["Z1S", "Z0p5S", "Z2S"]
chemZ_R8_new = ["Z1",  "Z05", "Z2"]
CO_lines = [1, 2] #CO (1-0), CO (2-1)
#R8_4pc_newacc
ivtks_R8_4pc = np.arange(200, 410, 10)
id_R8_4pc_old = "R8_4pc_newacc"

In [26]:
def copy_MHD(ivtks, id_old, id_new, dir_MHD_old, dir_new):
    """Copy Athena 4.2 MHD TIGRESS simulation output to data storage.
    input:
        ivtks: array or list of integers specifying vtk index
        id_old: string, old output id, e.g. R8_2pc_rst
        id_new: string, new output id, e.g. R8_2pc
        dir_MHD_old: string, old folder for data storage
        dir_new: string, new folder for data storage"""
    fn_par_old = "{:s}{:s}/{:s}.par".format(dir_MHD_old, id_old, id_old)
    fn_par_new = "{:s}{:s}/input/MHD/{:s}.par".format(dir_new, id_new, id_new)
    copy_file(fn_par_old, fn_par_new)
    fn_hst_old = "{:s}{:s}/hst/{:s}.hst".format(dir_MHD_old, id_old, id_old)
    fn_hst_new = "{:s}{:s}/history/MHD/{:s}.hst".format(dir_new, id_new, id_new)
    copy_file(fn_hst_old, fn_hst_new)
    for ivtk in ivtks:
        dir_MHD_new = "{:s}{:s}/{:04d}/MHD/".format(dir_new, id_new, ivtk)
        fn_vtk_old = "{:s}{:s}/vtk/{:s}.{:04d}.vtk".format(dir_MHD_old, id_old, id_old, ivtk)
        fn_vtk_new = "{:s}{:s}.{:04d}.vtk".format(dir_MHD_new, id_new, ivtk)
        copy_file(fn_vtk_old, fn_vtk_new)
    return

def copy_chemistry(ivtks, id_old, id_new, dir_chem_old, dir_new, chemZ_old, chemZ_new):
    """Copy Athena++ post-processing chemistry HDF5 output to data storage.
    input:
        ivtks: array or list of integers specifying vtk index
        id_old: string, old output id, e.g. R8_2pc_rst
        id_new: string, new output id, e.g. R8_2pc
        dir_chem_old: string, old folder for data storage
        dir_new: string, new folder for data storage
        chemZ_R8_old: list of strings, old names for metallicity
        chemZ_R8_new: list of strings, new names for metallicity"""
    for iZ in np.arange(len(chemZ_old)):
        Z_old = chemZ_old[iZ]
        Z_new = chemZ_new[iZ]
        for ivtk in ivtks:
            dir_chem_new = "{:s}{:s}/{:04d}/chem/".format(dir_new, id_new, ivtk)
            fn_old = "{:s}{:s}/{:s}/{:04d}/{:s}.out1.00001.athdf".format(
                dir_chem_old, id_old, Z_old, ivtk, id_old)
            fn_new = "{:s}{:s}/{:s}-{:s}.{:04d}.athdf".format(
                dir_chem_new, Z_new, id_new, Z_new, ivtk)
            copy_file(fn_old, fn_new)
            fn_xdmf_old = "{:s}{:s}/{:s}/{:04d}/{:s}.out1.00001.athdf.xdmf".format(
                dir_chem_old, id_old, Z_old, ivtk, id_old)
            fn_xdmf_new = "{:s}{:s}/{:s}-{:s}.{:04d}.athdf.xdmf".format(
                dir_chem_new, Z_new, id_new, Z_new, ivtk)
            copy_file(fn_xdmf_old, fn_xdmf_new)
    return

def copy_CO(ivtks, id_old, id_new, dir_CO_old, dir_new, chemZ_old, chemZ_new, lines):
    """Copy RADMC-3D post-processing CO lines PPV output to data storage.
    input:
        ivtks: array or list of integers specifying vtk index
        id_old: string, old output id, e.g. R8_2pc_rst
        id_new: string, new output id, e.g. R8_2pc
        dir_CO_old: string, old folder for data storage
        dir_new: string, new folder for data storage
        chemZ_R8_old: list of strings, old names for metallicity
        chemZ_R8_new: list of strings, new names for metallicity
        lines: array/list of integers, upper level of CO rotational lines.
               e.g. 1: J=1-0"""
    for iZ in np.arange(len(chemZ_old)):
        Z_old = chemZ_old[iZ]
        Z_new = chemZ_new[iZ]
        for ivtk in ivtks:
            dir_CO_new = "{:s}{:s}/{:04d}/CO_lines/".format(dir_new, id_new, ivtk)
            for iline in lines:
                fn_old = "{:s}{:s}/{:s}/il{:d}/{:04d}/image.bout".format(
                        dir_CO_old, id_old, Z_old, iline, ivtk)
                fn_new = "{:s}{:s}/il{:d}/{:s}-{:s}.il{:d}.{:04d}.bout".format(
                        dir_CO_new, Z_new, iline, id_new, Z_new, iline, ivtk)
                copy_file(fn_old, fn_new)
    return

def copy_rad(ivtks, id_old, id_new, dir_rad_old, dir_new):
    """Copy Athena 4.2 MHD TIGRESS simulation output 
       with radiaiton post-processing to data storage.
       Delete radiation fields.
    input:
        ivtks: array or list of integers specifying vtk index
        id_old: string, old output id, e.g. R8_2pc_rst
        id_new: string, new output id, e.g. R8_2pc
        dir_rad_old: string, old folder for data storage
        dir_new: string, new folder for data storage"""
    fields_del = ['rad_energy_density0','rad_energy_density1']
    for ivtk in ivtks:
        ivtk_old = int((ivtk-ivtks[0])/10 + 1)
        dir_rad_new = "{:s}{:s}/{:04d}/MHD_PI/".format(dir_new, id_new, ivtk)
        fn_vtk_old = "{:s}{:s}/vtk/{:s}.{:04d}.vtk".format(dir_rad_old, id_old, id_old, ivtk_old)
        fn_vtk_new = "{:s}{:s}.{:04d}.vtk".format(dir_rad_new, id_new, ivtk)
        delete_fields_vtk.delete_fields_vtk(fn_vtk_old, fn_vtk_new, fields_del, verbose=False)
    return

In [27]:
#R8_2pc_rst
copy_MHD(ivtks_R8_2pc, id_R8_2pc_old, "R8_2pc", dir_MHD_old, dir_new)
copy_chemistry(ivtks_R8_2pc, id_R8_2pc_old, "R8_2pc", dir_chem_old, dir_new, chemZ_R8_old, chemZ_R8_new)
copy_CO(ivtks_R8_2pc, id_R8_2pc_old, "R8_2pc", dir_CO_old, dir_new, chemZ_R8_old, chemZ_R8_new, CO_lines)
copy_rad(ivtks_R8_2pc, id_R8_2pc_old, "R8_2pc", dir_rad_old, dir_new)

In [22]:
#R8_4pc_newacc
copy_MHD(ivtks_R8_4pc, id_R8_4pc_old, "R8_4pc", dir_MHD_old, dir_new)
copy_rad(ivtks_R8_4pc, id_R8_4pc_old, "R8_4pc", dir_rad_old, dir_new)